In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, Row
from pyspark.sql.functions import *

In [2]:
spark = SparkContext(master='local', appName='DataFrames')
sqlContext = SQLContext(spark)

22/10/01 16:40:25 WARN Utils: Your hostname, debian resolves to a loopback address: 127.0.1.1; using 192.168.100.4 instead (on interface eno2)
22/10/01 16:40:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/10/01 16:40:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
path_file = '/home/jeo/Documents/platzi/curso_de_fundamentos_de_spark_para_big_data/curso-apache-spark-platzi-master/files/'
def eliminar_encabezado(indice, iterador):
    return iter(list(iterador)[1:])

In [4]:
deportistaOlimpicoRDD = spark.textFile(path_file+'deportista.csv') \
                        .map(lambda l: l.split(","))
deportistaOlimpicoRDD2 = spark.textFile(path_file+'deportista2.csv') \
                        .map(lambda l: l.split(","))

deportistaOlimpicoRDD = deportistaOlimpicoRDD.union(deportistaOlimpicoRDD2)

deportistaOlimpicoRDD = deportistaOlimpicoRDD.mapPartitionsWithIndex(eliminar_encabezado)

deportistaOlimpicoRDD = deportistaOlimpicoRDD.map(lambda l: (
    int(l[0]),
    str(l[1]),
    int(l[2]),
    int(l[3]),
    int(l[4]),
    float(l[5]),
    int(l[6])
))

deportista_schema = StructType([
    StructField('deportista_id', IntegerType(), False),
    StructField('nombre', StringType(), False),
    StructField('genero', IntegerType(), False),
    StructField('edad', IntegerType(), False),
    StructField('altura', IntegerType(), False),
    StructField('peso', FloatType(), False),
    StructField('equipo_id', IntegerType(), False)
])

deportistaDF = sqlContext.createDataFrame(deportistaOlimpicoRDD, deportista_schema)
deportistaDF.show(5)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0| 0.0|      278|
|            5|Christine Jacoba ...|     2|  21|   185|82.0|      705|
+-------------+--------------------+------+----+------+----+---------+
only showing top 5 rows



In [5]:
resultadoSchemaRDD = StructType([
    StructField('resultado_id', IntegerType(), False),
    StructField('medalla', StringType(), False),
    StructField('deportista_id', IntegerType(), False),
    StructField('juego_id', IntegerType(), False),
    StructField('evento_id', IntegerType(), False)
])
resultadoDF = sqlContext.read.schema(resultadoSchemaRDD).option('header', 'True').csv(path_file + 'resultados.csv')
resultadoDF.show(5)

+------------+-------+-------------+--------+---------+
|resultado_id|medalla|deportista_id|juego_id|evento_id|
+------------+-------+-------------+--------+---------+
|           1|     NA|            1|      39|        1|
|           2|     NA|            2|      49|        2|
|           3|     NA|            3|       7|        3|
|           4|   Gold|            4|       2|        4|
|           5|     NA|            5|      36|        5|
+------------+-------+-------------+--------+---------+
only showing top 5 rows



In [6]:
paises_schema = StructType([
    StructField('id_equipo', IntegerType(), False),
    StructField('equipo', StringType(), False),
    StructField('sigla', StringType(), False)
])
paisesDF = sqlContext.read.schema(paises_schema).option('header', 'True').csv(path_file + 'paises.csv')
paisesDF.show(5)

+---------+--------------------+-----+
|id_equipo|              equipo|sigla|
+---------+--------------------+-----+
|        1|         30. Februar|  AUT|
|        2|A North American ...|  MEX|
|        3|           Acipactli|  MEX|
|        4|             Acturus|  ARG|
|        5|         Afghanistan|  AFG|
+---------+--------------------+-----+
only showing top 5 rows



22/10/01 16:40:30 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, equipo, sigla
 Schema: id_equipo, equipo, sigla
Expected: id_equipo but found: id
CSV file: file:///home/jeo/Documents/platzi/curso_de_fundamentos_de_spark_para_big_data/curso-apache-spark-platzi-master/files/paises.csv


In [7]:
resultadoDF.registerTempTable('resultado')
deportistaDF.registerTempTable('deportista')
paisesDF.registerTempTable('paises')

In [8]:
sqlContext.sql("SELECT * FROM deportista").show(5)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0| 0.0|      278|
|            5|Christine Jacoba ...|     2|  21|   185|82.0|      705|
+-------------+--------------------+------+----+------+----+---------+
only showing top 5 rows



In [10]:
sqlContext.sql("""
    SELECT medalla, equipo, sigla FROM resultado r
    JOIN deportista d
    ON r.deportista_id = d.deportista_id
    JOIN paises p
    ON p.id_equipo = d.equipo_id
    WHERE medalla <> 'NA'
    ORDER BY sigla DESC
""").show()

22/10/01 16:43:35 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, equipo, sigla
 Schema: id_equipo, equipo, sigla
Expected: id_equipo but found: id
CSV file: file:///home/jeo/Documents/platzi/curso_de_fundamentos_de_spark_para_big_data/curso-apache-spark-platzi-master/files/paises.csv


+-------+--------+-----+
|medalla|  equipo|sigla|
+-------+--------+-----+
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
| Bronze|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
+-------+--------+-----+
only showing top 20 rows



In [14]:
spark

<SparkContext master=local appName=DataFrames>